In [1]:
#################################
#           battery EV          #
#################################

__version__ = '0.3'

import pandas as pd
import numpy as np
from datetime import timedelta


In [2]:
class price_profiles:
    """
     Input
    ----
    request status from the buildings 
    
    Return
    ----
    The Price Profiles
    """
    def __init__(self,request_status):
        self.request_status = request_status
    
    def getProfiles(self):
        import_price1 = pd.read_csv('data/octopus_imprt_price_southern.csv', index_col=0,parse_dates=True,header = 0)
        export_price1 = pd.read_csv('data/octopus_export_price_southern.csv', index_col=0,parse_dates=True,header = 0)

        start_time = "2019-1-31 00:00:00"
        end_time = "2019-5-15 22:30:00"
        
        import_price2 =import_price1.loc[start_time:end_time]
        export_price2 =export_price1.loc[start_time:end_time]
        
        import_price3 = import_price2.between_time('09:00:00','16:30:00')
        export_price3 = export_price2.between_time('09:00:00','16:30:00')
        
        for column in import_price3 [['Import Rate (p/kWh)']]:
            import_price = import_price3[column] 
            import_price_value = import_price.values
            
        for column in export_price3 [['Outgoing Agile Rate (p/kWh)']]:
            export_price = export_price3[column]
            export_price_value = export_price.values 
            
        return import_price_value, export_price_value 

In [3]:
class control_class:
    """
     Input
    ----
    request status from the buildings 
    
    Return
    ----
    a control signal
    """
    def __init__(self,request_status):
        self.request_status = request_status
        
    def signalGen(self,import_price_value,export_price_value, max_buy_price, min_sell_price):     
        T = len(import_price_value) 
        signal = np.zeros((T, 1))
        
        if self.request_status == 0:  
            for time,price in enumerate(import_price_value):
                if price < max_buy_price:   # Charge/Buys when price < Maximum_Buy_Price
                    signal[time] = 1
                else:
                    signal[time] = 0  # Set all other cases to "do nothing" first
                    
            for time,price in enumerate(export_price_value):
                if price > min_sell_price :  # Discharge/Sells when price > Minimum_Sell_Price 
                    signal[time] = -1 # Corrects some cases to sell electricity 
                
        elif self.request_status == 1: 
            signal = np.zeros((T, 1))  # Always connected to the buildings in this verison 
                   
        return signal

In [4]:
class EV_Component:
    """EV Component base class"""
    def __init__(self):
        self.dispatch_type = "EV Component"
        self.capacity = 0
        self.install_cost = 0
        self.depreciation_cost = 0
        self.lifetime = 15    # Assume all components have a lifespan of 15 years 

In [5]:
class EV_battery(EV_Component):
    """
    EV battery class
    Input
    -----
    Capacity : float
        Battery capacity, kWh.

    power : float
        Maximum power, kW.

    eff : float
        Charging/discharging efficiency between 0-1.

    dt : float
        Time interval (hours)

    T : int
        Number of intervales
    """
    def __init__(self, capacity, power, efficiency, dt, install_cost, signal):
        super().__init__()
        self.asset_type = 'EV Battery'
        self.capacity = capacity
        self.power = power * dt   # Convert kW to kWh
        self.eff = efficiency
        self.soc = np.ones(signal.size) * self.capacity
        self.install_cost = install_cost * 100  # p/kWh
        self.depreciation_cost = install_cost/self.lifetime
        self.signal = signal
        
    def getOutput(self, signal):
        """
        Battery control of charging/discharging in response to the signal

        Input
        -----
        Signal : numpy array

        Returns
        -------
        Battery energy use profile in kWh for each time interval : numpy array 
        """
  
        T = len(signal)
        output = np.zeros((T, 1))   # zero array of T rows, 1 column
        
        for time, signal_status in enumerate(signal):
            if time % 16 == 0:           # Reset the soc every morning at 9 am  
                soc = self.capacity - 5  # Capacity minus the energy used for commuting to work 
            else:
                soc = self.soc[time - 1]           
            if signal[time] == -1 and self.soc[time-1]>5:       # Signal indicating selling/ Discharging 
                                                                # Whilst also saving enough for traveling home
                output[time] = min(self.power, self.eff*soc)
                self.soc[time] = soc - (1/self.eff)*output[time]
            elif signal[time] == 1:  # Signal indicating buying/ Charging 
                output[time] = max(-self.power, - (1/self.eff) * (self.capacity - soc))
                self.soc[time] = soc - self.eff * output[time]        
            else:  # do nothing
                self.soc[time] = soc
            print(self.soc[time])
        return output 
    
    def getGridCost(self, output, import_price_value, export_price_value,request_status):
        """
        Input
        -----
        Signal : numpy array

        Returns
        -------
        Total Grid Cost
        """
  
        cost_arr_size = len(output)
        cost_array = np.zeros((cost_arr_size, 1))   # zero array of T rows, 1 column
        
        for time, output_power in enumerate(output):
            if request_status == 0:
                if output[time] > 0:    # Positive output power, selling
                    cost_array[time] = - export_price_value[time]*output[time] # Cost is negative
                elif output[time] < 0:  # Negative output power, buying
                    cost_array[time] = - import_price_value[time]*output[time] # Cost is positive           
                elif output[time] == 0:  # No power output, do nothing 
                    cost_array[time] = 0         
                #print(cost_array[time])
                
            if request_status == 1:
                cost_array[time] = 0             
        grid_cost = np.sum(cost_array) # in pence 
        grid_cost_GBP = grid_cost/100
        print('Grid cost: £ %3.3f' % grid_cost_GBP)
        return grid_cost_GBP
    

In [6]:
class V2G_Charger(EV_Component): 
    def __init__(self, install_cost = 0):
        super().__init__()
        self.asset_type = 'EV Charger'
        self.install_cost = install_cost * 100  # p/kWh

In [7]:
#######################
##   Main Programme  ##
#######################
dt = 30/60 # Half-hour intervals 
request_status = 0 # Buildings require electricity status

max_buy_price = 4
min_sell_price = 5

In [8]:
# Price 
price_profiles1 = price_profiles(request_status)
import_price_value, export_price_value = price_profiles1.getProfiles()

# Control Signals
signal = control_class(request_status)
signal1 = signal.signalGen(import_price_value, export_price_value, max_buy_price, min_sell_price)

In [9]:
# Battery
battery_capacity = 60  # kWh
battery_power = 10  # kW
battery_eff = 0.8
battery_cost = 0 
battery_site1 = EV_battery(battery_capacity,battery_power,battery_eff, dt, battery_cost,signal1)
output1 = battery_site1.getOutput(signal1)

48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
42.5
36.25
30.0
23.75
17.5
17.5
11.25
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
48.75
4

In [10]:
grid_cost =  battery_site1.getGridCost(output1, import_price_value, export_price_value,request_status)

Grid cost: £ -227.241
